In [3]:
import pandas as pd
import numpy as np

from Ames_Functions import * #imports OSMR, get_geoapify, get_dist, locator, find_min_dist, load_state_pkl, save_state_pkl, EDA_report, etc.

from geopy import Nominatim

import pickle

import os
pd.set_option("display.max_columns", None)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## House Latlongs Lookup Section

In [4]:
#Loading Datasets
df_RE_latest = pd.read_csv("data/Ames_Real_Estate_Data_Latest.csv")
df_EssentialLatLong = pd.read_csv("data/df_EssentialLatLong_v2.csv")

#2021
df_dist_time_all2021 = load_state_pkl()['2021dfs_for_feature_engineering'][0]
df_RE_latlongs2021 = load_state_pkl()['df_RE_latlong2021'][0]

#2019
df_dist_time_all2019 = load_state_pkl()['2019dfs_for_feature_engineering'][0]
df_RE_latlongs2019 = load_state_pkl()['df_RE_latlong2019'][0]

/var/folders/2n/5k95xl754cb_szsgjkmj33k80000gn/T/ipykernel_74387/473993362.py:2: DtypeWarning: Columns (36,40,81,87) have mixed types. Specify dtype option on import or set low_memory=False.
  df_RE_latest = pd.read_csv("data/Ames_Real_Estate_Data_Latest.csv")
/Users/lelandmurrin/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DummyRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/lelandmurrin/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_per

In [ ]:
#Generating and then storing Latlongs using Geoapify

#2019
df_RE_latlongs2019 = df_RE_latest.query('YrSold_YYYY == 2019').drop_duplicates()
latlongs2019 = df_RE_latlongs2019.Prop_Addr.apply(lambda x: get_geoapify(x + ", Ames, Iowa"))

#2021
df_RE_latlongs2021 = df_RE_latest.query('YrSold_YYYY == 2021').drop_duplicates()
latlongs2021 = df_RE_latlongs2021.Prop_Addr.apply(lambda x: get_geoapify(x + ", Ames, Iowa"))

In [25]:
locator = Nominatim(user_agent="myGeocoder")

def create_cleanup_latlongs_csv(latlongs, filename, df_RE_latlongs):
    # Checking Latlongs with reverse geocoding
    s_address = latlongs.apply(lambda x: locator.reverse(x).raw['address'])

    # Pulling just the road from the reverse lookup address
    s_road = pd.json_normalize(s_address).road.fillna("Monroe Drive")
    s_road.to_frame()

    # Adding latlongs to the housing dataframe
    df_RE_latlongs = df_RE_latlongs.join(latlongs.str.split(",", expand = True).set_axis(["Lat", "Long"], axis = 1))

    # Adding reverse_lookup_rd field to dataframe for comparison to PA-Strt for manual corrections. 
    # This allowed for identifying discrepancies in the reverse_lookup_addresses and addresses listed in the dataset
    df_RE_latlongs['reverse_lookup_rd'] = s_road.values
    mask = df_RE_latlongs.apply(lambda x: x["PA-Strt"].lower() not in x.reverse_lookup_rd.lower(), axis = 1)
    df_RE_latlongs.query('@mask')[["Prop_Addr", "reverse_lookup_rd", "Lat", "Long"]].to_csv(filename)
    return df_RE_latlongs

In [ ]:
#2021
df_RE_latlongs2021 = create_cleanup_latlongs_csv(latlongs2021, "df_RE_latlongs2021_corrections.csv", df_RE_latlongs2021)

#2019
df_RE_latlongs2019 = create_cleanup_latlongs_csv(latlongs2019, "df_RE_latlongs2019_corrections.csv", df_RE_latlongs2019)

In [16]:
df_RE_latest.filter(regex = "Oth")

,OthAc_S
0,0
1,0
2,0
3,288800
4,0
...,...
22958,0
22959,0
22960,0
22961,0


In [147]:
#Read in manual corrections file

def update_latlongs_corrections(filename, df_RE_latlongs):
    df_latlong_corrections = pd.read_csv(filename, index_col = 0)

    #updating corrections to the dataset
    df_RE_latlongs.loc[df_latlong_corrections.index, ["Lat", "Long"]] = df_latlong_corrections[["Lat", "Long"]]
    return df_RE_latlongs

#2021
df_RE_latlongs2021 = update_latlongs_corrections("df_RE_latlongs2021_corrections.csv", df_RE_latlongs2021)

#2019
df_RE_latlongs2019 = update_latlongs_corrections("df_RE_latlongs2019_corrections.csv", df_RE_latlongs2019)

,MapRefNo,GeoRefNo,Tier,Range,Prop_Addr,ZngCdPr,ZngCdSc,ZngOLPr,ZngOLSc,ClassPr_S,...,PA-StSfx,PA-PostD,PA-UnTyp,PA-UntNo,Date,Source,NmbrBRs,reverse_lookup_rd,Lat,Long
11371,906329170.0,906329170,0,0,5116 SPRINGBROOK CIR,FS-RL,NaN,NaN,NaN,RESIDENTIAL,...,CIR,NaN,NaN,NaN,31-Mar-22,Ames City Assessor,2.0,6th Street,42.028568,-93.687674
11373,906329180.0,906329180,0,0,5122 SPRINGBROOK CIR,FS-RL,NaN,NaN,NaN,RESIDENTIAL,...,CIR,NaN,NaN,NaN,31-Mar-22,Ames City Assessor,2.0,6th Street,42.028348,-93.687718


In [150]:
#Saving latlongs dataframe to the master dictionary stored in Ames_notebook_state pkl file to use accross multiple notebooks

save_state_pkl('df_RE_latlong2021',
               df_RE_latlongs2021,
               "latlong lookup for 2021 using geoapify and reverse lookup using nominatim with corrections done by hand"
               )

save_state_pkl('df_RE_latlong2019',
               df_RE_latlongs2019,
               "latlong lookup for 2019 using geoapify and reverse lookup using nominatim with corrections done by hand"
               )

## Driving time Lookups Section

In [44]:
#Finding closest business per service to each house and creating dataframe containing that information (df_subset)

def create_df_subset (df_RE_latlongs, df_EssentialLatLong):
    #the same code is found inside the Ames_Functions file, but returns df_subset with extra processing via pivot etc.
    df_cross = df_RE_latlongs.join(df_EssentialLatLong, how = "cross", lsuffix = '_house', rsuffix = "_biz")
    df_cross["dist"] = df_cross.apply(lambda x: get_dist(np.array([x.Lat_house, x.Long_house]), np.array([x.Lat_biz, x.Long_biz])), axis = 1)
    df_subset = df_cross.loc[df_cross.groupby(["Service", "SaleID"]).dist.idxmin()]
    return df_subset

In [46]:
#This cell takes a while to run (1hour+)

#Looking up drive times/distances for the closest businesses per service to each house and storing them in from_biz dictionary

def drive_time_dist_lookups(df_subset):
    from_biz = {}
    OSMR_list = df_subset.Service.unique()

    for i in OSMR_list:
        from_biz[i] = df_subset.query(f"Service == '{i}'").apply(lambda x: OSMR(x.Lat_biz, x.Long_biz, x.Lat_house, x.Long_house), axis = 1)
        
    return pd.DataFrame(from_biz)

In [58]:
def create_driv_time_final_dict(from_biz_final, df_subset, df_RE_latlongs):
        from_biz_stacked = from_biz_final.stack().reset_index().set_axis(["df_subset_index", "Service", "time_dist"], axis = 1)

        from_biz_temp = from_biz_stacked.join(from_biz_stacked
                                                .apply(lambda x: f"{x.time_dist[0][0]}, {x.time_dist[0][1]}", axis = 1)
                                                .str.split(", ", expand = True)
                                                .set_axis(["time_secs", "driving_dist_meters"], axis = 1)
                                                .apply(lambda x: x.astype(float))
                                                ).drop("time_dist", axis = 1)

        driv_final_dict = {}

        for k, f in zip(["from_time", "from_driv", "dist"], ["time_secs", "driving_dist_meters", "dist"]):
                driv_final_dict[k] = (df_subset
                                        .join(from_biz_temp.set_index("df_subset_index"), rsuffix = "_driv")
                                        .query("Service == Service_driv")
                                        .drop("Service_driv", axis = 1)
                                        .pivot(index = ["SaleID", "Lat_house", "Long_house"], columns = "Service", values = f)
                                        .reset_index()
                                        .set_index("SaleID").join(df_subset[["SaleID", "SalePrice"]].drop_duplicates().set_index("SaleID").SalePrice)
                                )

        driv_final_dict['df_min_time'] = (df_subset.join(from_biz_temp.set_index("df_subset_index"), rsuffix = "_driv")
            .drop("Service_driv", axis = 1)
            .pivot(index = ["SaleID"], columns = "Service", values = "time_secs")
            .reset_index()
            .set_index("SaleID")
            .add_suffix("_time")
            )
        
        driv_final_dict['df_min_dist'] = find_min_dist(df_RE_latlongs, df_EssentialLatLong).set_index("SaleID")
                             

        return driv_final_dict

In [99]:
def create_df_for_feature_engineering(driv_final_dict, df_subset):
    def service_pivot (left_df, right_df, v):
        return left_df.copy().join(right_df.pivot(columns = "Service", index = "SaleID", values = v).add_suffix("_" + v))

    df_min_time = driv_final_dict['df_min_time']
    df_min_dist = driv_final_dict['df_min_dist']

    return (df_min_dist
                .join(df_min_time)
                .pipe(service_pivot, df_subset, "Lat_biz")
                .pipe(service_pivot, df_subset, "Long_biz")
                .drop("SalePrice", axis = 1)
                .join(df_min_dist.SalePrice)
            )

## Creating 2021 dfs after API drive time/distance lookups in preparation for feature engineering 

In [49]:
#Creating df_subset
df_subset2021 = create_df_subset(df_RE_latlongs2021, df_EssentialLatLong)

In [50]:
#creating from_biz_final
from_biz_final2021 = drive_time_dist_lookups(df_subset2021)
from_biz_final2021

#save_state_pkl('from_biz_api2021', from_biz_final2021, "Adding 2021 services driving times/distances lookups from OSMR ")

,Arts,Elem_School,Golf,Gym,High_School,Historic,ISU,Library,Medical,Movies,Organic_Groceries,Park,Recreation,Religion,Restaurant,Shopping,Spa,Sports_Venue
1,"[[633.8, 6846.7]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,"[[340.5, 4066.3]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,"[[131.1, 1488.6]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,"[[288.4, 3129.8]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,"[[504.8, 5289.6]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25943,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[282.9, 4091.9]]",NaN,NaN
25945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[363.9, 4665.5]]",NaN
25947,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[637.1, 7927.1]]"
25949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[375.7, 4907.1]]",NaN,NaN,NaN


In [ ]:
#Creating driv_final_dict
driv_final_dict2021 = create_driv_time_final_dict(from_biz_final2021, df_subset2021, df_RE_latlongs2021)

In [ ]:
#Creating final output dataframe for feature engineering to store in Ames_notebook_state pkl file
df_for_feature_engineering2021 = create_df_for_feature_engineering(driv_final_dict2021, df_subset2021)

## Creating 2019 dfs after API drive time/distance lookups in preparation for feature engineering 

In [52]:
#Creating df_subset
df_subset2019 = create_df_subset(df_RE_latlongs2019, df_EssentialLatLong)

In [54]:
#creating from_biz_final
from_biz_final2019 = drive_time_dist_lookups(df_subset2019)
from_biz_final2019

#save_state_pkl('from_biz_api2019', from_biz_final2019, "Adding 2019 services driving times/distances lookups from OSMR ")

,Arts,Elem_School,Golf,Gym,High_School,Historic,ISU,Library,Medical,Movies,Organic_Groceries,Park,Recreation,Religion,Restaurant,Shopping,Spa,Sports_Venue
2,"[[675.6, 8106.8]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,"[[440.1, 4269.8]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,"[[389.7, 2898.7]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,"[[441.3, 3925.6]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,"[[656.6, 7138.7]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[72.8, 717.2]]",NaN,NaN
36933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[153.8, 1290.8]]",NaN
36935,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[427.6, 3961.7]]"
36937,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[165.6, 1532.4]]",NaN,NaN,NaN


In [63]:
#Creating driv_final_dict
driv_final_dict2019 = create_driv_time_final_dict(from_biz_final2019, df_subset2019, df_RE_latlongs2019)

In [ ]:
#Creating final output dataframe for feature engineering to store in Ames_notebook_state pkl file
df_for_feature_engineering2019 = create_df_for_feature_engineering(driv_final_dict2019, df_subset2019)

## Final Notebook Output

In [272]:
save_state_pkl("2019dfs_for_feature_engineering",
               df_for_feature_engineering2019,
               "After data transformation of the driving time/distance API results in preparation for feature engineering. Also includes business information and point to point distances."
                )

save_state_pkl("2021dfs_for_feature_engineering",
               df_for_feature_engineering2021,
               "After data transformation of the driving time/distance API results in preparation for feature engineering. Also includes business information and point to point distances."
                )